In [1]:
%reload_ext autoreload
%autoreload 2

setup_already_done = False
# prj_dir_name = 'VideoPrediction_MovingMNIST'

In [2]:
if not setup_already_done:
    %cd ..
    %pip install -qr requirements.txt --quiet
    
    setup_already_done = True
# ! wget -q https://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy

/home/silverio/Projects/uni/DeepLearning/VideoPrediction_MovingMNIST
Note: you may need to restart the kernel to use updated packages.


In [3]:
!nvidia-smi

Tue Jun 27 18:07:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P3    N/A /  N/A |      4MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import optim, nn, utils, Tensor
import torchvision
from torchvision.transforms import ToTensor
import lightning.pytorch as pl



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


device(type='cuda')

In [5]:
from src.MovingMNIST import MovingMNIST

dataset = MovingMNIST(data_path="data/mnist_test_seq.npy")

# Visualization Test
dataset.visualize(0)


In [6]:
loader = DataLoader(dataset)

In [7]:
from src.simpleLSTM import simpleLSTM

model = simpleLSTM()


In [8]:
for data in loader:
    model.training_step(data,0)
    break

In [9]:
trainer = pl.Trainer(limit_train_batches=5, max_epochs=1)
trainer.fit(model=model, train_dataloaders=loader)

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | loss_fn | MSELoss    | 0     
1 | relu    | ReLU       | 0     
2 | encoder | Sequential | 4.2 M 
3 | lstm    | LSTM       | 3.1 M 
4 | decoder | Sequential | 4.7 M 
---------------------------------------
12.1 M    Trainable params
0         Non-trainable params
12.1 M    Total params
48.275    Total estimated model params size (MB)
/home/sil

Training: 0it [00:00, ?it/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)